# Wikipedia typos

Found about this [here](https://www.dcs.bbk.ac.uk/~ROGER/corpora.html).

In [10]:
%load_ext autoreload
%autoreload 2

In [1]:
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/Wikipedia:Lists_of_common_misspellings/For_machines'

content = requests.get(url).content.decode()
soup = BeautifulSoup(content)

In [9]:
from collections import defaultdict

typos = defaultdict(list)

for line in soup.find(name='pre').text.splitlines():
    typo, correct = line.split('->')
    typos[correct].append(typo)

typos = dict(typos)

print(f'{len(typos):,d} words')
print(f'{sum(map(len, typos.values())):,d} misspellings')

3,156 words
4,291 misspellings


In [52]:
import orc
from dataclasses import dataclass
import regex

for correct, misspellings in typos.items():
    for misspelling in misspellings:
        
        fuzzy_pattern = f"({correct})" + "{s<=3,i<=3,d<=3}"
        for m in regex.finditer(
            fuzzy_pattern, misspelling, regex.BESTMATCH
        ):
            near_match = orc.NearMatch.from_regex(m, correct)
            assert near_match.edits.do(m.group()) == correct